In [15]:
import sys
sys.path.append("/home/schmid/Text2Head/NPHM")

import torch
from torch.optim import Adam
from utils.pipeline import forward, get_latent_mean_std, get_latent_from_text
import optuna

torch.cuda.empty_cache()

This notebook serves to find the best hyperparameters for the optimization.
Important:
- to make different runs comparable, we always start from lat_mean
- for the model to be able to yield good results for varying prompts, we use 3 prompts from different categories (gender, ethnicity, hairstyle) for the hyperparameter tuning

In [16]:
lat_mean, lat_std = get_latent_mean_std()

hparams = {
        'resolution': 120,
        'n_iterations': 50,
        'optimizer_lr': 0.02,
        'lr_scheduler_factor': 0.1,
        'lr_scheduler_patience': 2,
        'lr_scheduler_min_lr': 1e-5
    }

prompts = []
prompts.append('A woman')
prompts.append('An Asian person')
prompts.append('A person with beard')

def latent_optimization(hparams):
    all_CLIP_scores = []
    for prompt in prompts:
        _, best_score, _ = get_latent_from_text(prompt, hparams, init_lat=lat_mean)
        all_CLIP_scores.append(best_score)

    all_CLIP_scores_tensor = torch.stack(all_CLIP_scores)
    print(all_CLIP_scores_tensor)
    # Calculate the average
    average_score = torch.mean(all_CLIP_scores_tensor)

    return average_score

def objective(trial):

    search_space = {
        'resolution': 120,
        'n_iterations': trial.suggest_categorical('n_iterations', [40, 50, 60, 70]),
        'optimizer_lr': trial.suggest_float('optimizer_lr', 1e-4, 1e-2, log=True),
        'lr_scheduler_factor': trial.suggest_float('lr_scheduler_factor', 0.1, 0.9),
        'lr_scheduler_patience': trial.suggest_int('lr_scheduler_patience', 2, 5),
        'lr_scheduler_min_lr': trial.suggest_float('lr_scheduler_minlr', 1e-5, 1e-4, log=True),
    }

    hparams.update(search_space)

    return latent_optimization(hparams)

In [17]:
if __name__ == '__main__':
    study = optuna.create_study(storage="sqlite:///../optuna_study_hparams.db", study_name="optim_hparams", direction='maximize', load_if_exists=True)
    study.optimize(objective, n_trials=1)
        
    best_params = study.best_params
    print(best_params)

[I 2023-12-07 16:29:32,364] Using an existing study with name 'optim_hparams' instead of creating a new one.


  0%|          | 0/50 [00:00<?, ?it/s]
[W 2023-12-07 16:29:32,530] Trial 1 failed with parameters: {'n_iterations': 50, 'optimizer_lr': 0.0037586281146446444, 'lr_scheduler_factor': 0.22516937725024164, 'lr_scheduler_patience': 2, 'lr_scheduler_minlr': 1.5171601286511644e-05} because of the following error: OutOfMemoryError('CUDA out of memory. Tried to allocate 440.00 MiB (GPU 0; 10.91 GiB total capacity; 8.29 GiB already allocated; 95.44 MiB free; 8.57 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF').
Traceback (most recent call last):
  File "/home/schmid/miniconda3/envs/nphm/lib/python3.9/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_66556/3085136350.py", line 43, in objective
    return latent_optimization(hparams)
  File "/tmp/ipykernel_66556/3085136350.p

OutOfMemoryError: CUDA out of memory. Tried to allocate 440.00 MiB (GPU 0; 10.91 GiB total capacity; 8.29 GiB already allocated; 95.44 MiB free; 8.57 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF